In [92]:
import pandas as pd

# Arquivos de entrada referente a chuva
taioChuva = pd.read_fwf('txt\83050000TAIÓ_2962005-1442014-chuva.txt', header=None)
ituporangaChuva = pd.read_fwf('txt\83250000ITUPORANGA_3172005-1442014-chuva.txt', header=None)
riodosulChuva = pd.read_fwf('txt\83300200RIODOSUL-NOVO_2862005-1442014-Chuva.txt', header=None)

#TRASNFORMAÇÃO CHUVA - Inserindo header com a data e a chuva, e removido o indice 0 do dataframe (que continha o cabeçalho do arquivo original) 
taioChuva.columns = ['Data', 'Chuva Taio']
taioChuva = taioChuva.drop(taioChuva.index[0])

ituporangaChuva.columns = ['Data', 'Chuva Ituporanga']
ituporangaChuva = ituporangaChuva.drop(ituporangaChuva.index[0])

riodosulChuva.columns = ['Data', 'Chuva Rio do Sul']
riodosulChuva = riodosulChuva.drop(riodosulChuva.index[0])


In [93]:
# Arquivos de entrada referente ao nível
taioNivel = pd.read_fwf('txt\83050000TAIÓ_2962005-1442014-nível.txt', header=None)
ituporangaNivel = pd.read_fwf('txt\83250000ITUPORANGA_2962005-1032014-nível.txt', header=None)
riodosulNivel = pd.read_fwf('txt\83300200RIODOSUL-NOVO_2982005-1422014 -nível.txt', header=None)

#TRASNFORMAÇÃO NIVEL - Inserindo header com a data e a chuva, e removido o indice 0 do dataframe (que continha o cabeçalho do arquivo original) 
taioNivel.columns = ['Data', 'Nivel Taio']
taioNivel = taioNivel.drop(taioNivel.index[0])

ituporangaNivel.columns = ['Data', 'Nivel Ituporanga']
ituporangaNivel = ituporangaNivel.drop(ituporangaNivel.index[0])

riodosulNivel.columns = ['Data', 'Nivel Rio do Sul']
riodosulNivel = riodosulNivel.drop(riodosulNivel.index[0])


In [94]:
#Juntando os dataframes afins de chuvas
chuva1 = pd.merge(taioChuva, ituporangaChuva, how='inner', on = 'Data')
chuva = pd.merge(chuva1, riodosulChuva, how='inner', on = 'Data')

#TRANSFORMAÇÃO CHUVA - Removendo as linhas que são nulas
chuva.dropna(inplace=True)


In [95]:
#Juntando os dataframes afins de niveis
nivel1 = pd.merge(taioNivel, ituporangaNivel, how='inner', on = 'Data')

nivel = pd.merge(nivel1, riodosulNivel, how='inner', on = 'Data')

#TRANSFORMAÇÃO NIVEL - Removendo as linhas que são nulas
nivel.dropna(inplace=True)


In [96]:
#Juntando os dataframes de chuvas e niveis
cheias = pd.merge(chuva, nivel, how = 'inner', on = 'Data')
cheias.to_csv('cheias.csv', index=False)
cheias.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 66957 entries, 0 to 66956
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Data              66957 non-null  object
 1   Chuva Taio        66957 non-null  object
 2   Chuva Ituporanga  66957 non-null  object
 3   Chuva Rio do Sul  66957 non-null  object
 4   Nivel Taio        66957 non-null  object
 5   Nivel Ituporanga  66957 non-null  object
 6   Nivel Rio do Sul  66957 non-null  object
dtypes: object(7)
memory usage: 4.1+ MB
